In [154]:
import numpy as np
from pyblock2._pyscf.ao2mo import integrals as itg
from pyblock2.driver.core import DMRGDriver, SymmetryTypes
from OrbitalRotation.tools.orbital_rotation_general import orbital_rotation_p_q, orbital_rotation_p_q_nf
from OrbitalRotation.tools.obt_to_tbt import get_obt_combined_tbt
import scipy
import pyscf
from basic_definitions import chemist_to_physicist
import dmrghandler.src.dmrghandler.dmrg_calc_prepare as dmrg_calc_prepare
from dmrghandler.src.dmrghandler.dmrg_looping import dmrg_central_loop
from dmrghandler.src.dmrghandler.qchem_dmrg_calc import single_qchem_dmrg_calc
from get_dmrg_config import get_dmrg_process_param, get_dmrg_config, get_dmrg_config_type1, get_custom_bd_dmrg_config

In [155]:
import shutil
import os

# Define the directory path
dir_path = "./mps_directory"

# Check if the directory exists
if os.path.exists(dir_path):
    # Delete the directory and all its contents
    shutil.rmtree(dir_path)
    print(f"Deleted '{dir_path}' and all its contents.")
else:
    print(f"Directory '{dir_path}' does not exist.")
    
# Define the directory path
dir_path = "./mps_directory2"

# Check if the directory exists
if os.path.exists(dir_path):
    # Delete the directory and all its contents
    shutil.rmtree(dir_path)
    print(f"Deleted '{dir_path}' and all its contents.")
else:
    print(f"Directory '{dir_path}' does not exist.")

Directory './mps_directory' does not exist.
Deleted './mps_directory2' and all its contents.


In [156]:
from pyscf import gto, scf

mol = gto.M(atom="O 0 0 0; H 0 1 0; H 0 0 1", basis="sto3g", verbose=0)
mf = scf.RHF(mol).run(conv_tol=1E-14)
ncas, n_elec, spin, ecore, h1e, g2e, orb_sym = itg.get_rhf_integrals(mf,
    ncore=0, ncas=None, g2e_symm=1)
print(f"n Cas: {ncas}")
print(f"orb sym: {orb_sym}")
driver = DMRGDriver(scratch="./tmp", symm_type=SymmetryTypes.SZ, n_threads=4)
driver.initialize_system(n_sites=ncas, n_elec=n_elec, spin=spin, orb_sym=orb_sym)

idx = driver.orbital_reordering(h1e, g2e)
h1e = h1e[idx][:, idx]
g2e = g2e[idx][:, idx][:, :, idx][:, :, :, idx]
orb_sym = np.array(orb_sym)[idx]

absorbed_tbt = pyscf.fci.direct_nosym.absorb_h1e(np.array(h1e.copy()), np.array(g2e.copy()), norb=ncas, nelec=n_elec)
h_in_tbt = pyscf.ao2mo.restore(1, absorbed_tbt.copy(), ncas).astype(np.array(h1e.copy()).dtype, copy=False)
combined_tbt_optimizing = 0.5 * h_in_tbt
obt_dummy = np.zeros((ncas, ncas))
obt, tbt = chemist_to_physicist(obt_dummy, 0.5 * h_in_tbt)

n Cas: 7
orb sym: [0, 0, 0, 0, 0, 0, 0]


In [157]:
dmrg_params = get_dmrg_config(
    ncas, n_elec, spin,
    2 * spin + 1)
loop_results = dmrg_central_loop(obt, tbt, dmrg_params, 1000, 1e6, 1e-300   , "./mps_directory", verbosity=1, move_mps_to_final_storage_path=None)

integral cutoff error =  0.0
mpo terms =       4672

Build MPO | Nsites =     7 | Nterms =       4672 | Algorithm = FastBIP | Cutoff = 1.00e-120
 Site =     0 /     7 .. Mmpo =    30 DW = 0.00e+00 NNZ =       30 SPT = 0.0000 Tmvc = 0.000 T = 0.005
 Site =     1 /     7 .. Mmpo =    70 DW = 0.00e+00 NNZ =      331 SPT = 0.8424 Tmvc = 0.000 T = 0.002
 Site =     2 /     7 .. Mmpo =   114 DW = 0.00e+00 NNZ =      599 SPT = 0.9249 Tmvc = 0.000 T = 0.002
 Site =     3 /     7 .. Mmpo =   114 DW = 0.00e+00 NNZ =     2241 SPT = 0.8276 Tmvc = 0.000 T = 0.003
 Site =     4 /     7 .. Mmpo =    70 DW = 0.00e+00 NNZ =      591 SPT = 0.9259 Tmvc = 0.000 T = 0.001
 Site =     5 /     7 .. Mmpo =    30 DW = 0.00e+00 NNZ =      331 SPT = 0.8424 Tmvc = 0.000 T = 0.001
 Site =     6 /     7 .. Mmpo =     1 DW = 0.00e+00 NNZ =       30 SPT = 0.0000 Tmvc = 0.000 T = 0.000
Ttotal =      0.016 Tmvc-total = 0.002 MPO bond dimension =   114 MaxDW = 0.00e+00
NNZ =         4153 SIZE =        33216 SPT = 0.8750

KeyboardInterrupt: 

In [158]:
def prepare_dmrg_config(bd):
    
    dmrg_par = get_custom_bd_dmrg_config(
        ncas, n_elec, spin,
        2 * spin + 1, bd)
    
    return dmrg_par

In [159]:
def get_anti_symmetric(n):
    """
    Construct list of anti-symmetric matrices kappa_pq based on n*(n-1)/2 
    """
    kappas = []
    for p in range(n):
        for q in range(p+1, n):
            kappa = np.zeros((n, n))
            kappa[p, q] = 1
            kappa[q, p] = -1
            kappas.append(kappa)
    return kappas

def construct_anti_symmetric(n, params):
    """
    Constrcut the nxn anti-symmetric matrix based on the sum of basis with oparams as coefficients
    """
    real_anti = get_anti_symmetric(n)
    anti_symm = np.zeros((n, n))
    for idx, antisym in enumerate(real_anti):
        anti_symm += params[idx] * antisym
    return anti_symm

def construct_orthogonal(n, params):
    """
    The parameters are n(n-1)/2 terms that determines the lower diagonals of e^{anti-symmetric}
    """
    anti_symm = construct_anti_symmetric(n, params)
    return scipy.linalg.expm(anti_symm)

In [160]:
def energy_cost_fn(params):
    U = construct_orthogonal(ncas, params)
    

    p = np.einsum_path('ak,bl,cm,dn,klmn->abcd', U, U, U, U, combined_tbt_optimizing.copy())[0]
    rotated_tbt = np.einsum('ak,bl,cm,dn,klmn->abcd', U, U, U, U, combined_tbt_optimizing.copy(), optimize = p)
    
    one_body_tensor, two_body_tensor = chemist_to_physicist(obt_dummy, rotated_tbt)
    bd = 1
    dmrg_param = prepare_dmrg_config(bd)
    
    result = single_qchem_dmrg_calc(one_body_tensor, two_body_tensor, dmrg_param)
    energy = result['dmrg_ground_state_energy']
    return energy

def diagonal_cost_fn(params):
    U = construct_orthogonal(ncas, params)
    

    p = np.einsum_path('ak,bl,cm,dn,klmn->abcd', U, U, U, U, combined_tbt_optimizing.copy())[0]
    rotated_tbt = np.einsum('ak,bl,cm,dn,klmn->abcd', U, U, U, U, combined_tbt_optimizing.copy(), optimize = p)
    total_sum = 0
    diag_sum = 0
    for p in range(ncas):
        for q in range(ncas):
            for r in range(ncas):
                for s in range(ncas):
                    total_sum += np.abs(rotated_tbt[p, q, r, s])
                    if p == q or r == s:
                        diag_sum += np.abs(rotated_tbt[p, q, r, s])
    
    return 1 - (diag_sum / total_sum)


def callback(xk):
    print(f"Current objective value: {energy_cost_fn(xk)}")

In [161]:
init_param = []
eig, eigv = scipy.linalg.eigh(h1e)
a = scipy.linalg.logm(eigv)
n = len(a[0])
print(n)
for p in range(n):
    for q in range(p+1, n):
        init_param.append(a[p, q])
print(scipy.linalg.expm(a) - eigv)

7
[[-4.15799926e-16  3.72845783e-16  0.00000000e+00  7.79328031e-16
   6.65784417e-17 -2.40539854e-16 -4.44089210e-16]
 [-1.37329560e-16  9.85638030e-17  1.11022302e-16  5.30222923e-16
   4.05825098e-17 -4.12289735e-16  3.33066907e-16]
 [ 1.01654796e-15  1.11022302e-16  4.53615524e-16  1.05471187e-15
  -4.52119399e-17 -1.11022302e-16 -7.07176609e-16]
 [ 9.95731275e-16 -1.87350135e-15 -5.27652153e-16  1.66533454e-15
  -8.02983474e-18  1.27675648e-15 -3.28960320e-16]
 [-1.03957076e-28  2.25756490e-18 -1.27584169e-16  1.57322541e-17
   0.00000000e+00 -9.04792997e-17 -3.29641284e-18]
 [-1.49186219e-16 -3.33066907e-16  4.53859036e-16 -1.11022302e-15
   1.93663365e-17  2.60902411e-15  8.60657295e-16]
 [-1.11022302e-16 -2.42861287e-17  1.93087438e-16 -5.03069808e-17
  -1.16862237e-19 -5.26488575e-16 -1.03956704e-15]]


In [162]:
import random
from scipy.optimize import minimize

# params = [random.random() for _ in range(int(ncas * (ncas - 1) / 2))]
res = minimize(energy_cost_fn, init_param, method='nelder-mead',
               options={'xatol': 1e-30, 'disp': True, 'maxfev': 10000}, callback=callback)

Current objective value: -83.36224107604369
Current objective value: -83.36224107604369
Current objective value: -83.36224107604369
Current objective value: -83.36224107604369
Current objective value: -83.36224107604369
Current objective value: -83.36224107604369
Current objective value: -83.36224107604369
Current objective value: -83.36224107604369
Current objective value: -83.36224107604369
Current objective value: -83.36224107604369
Current objective value: -83.36224107604369
Current objective value: -83.36224107604369
Current objective value: -83.36224107604369
Current objective value: -83.36224107604369
Current objective value: -83.36224107604369
Current objective value: -83.36224107604369
Current objective value: -83.36224107604369
Current objective value: -83.36224107604369
Current objective value: -83.41259768874659
Current objective value: -83.41259768874659
Current objective value: -83.41259768874659
Current objective value: -83.41259768874659
Current objective value: -83.412

KeyboardInterrupt: 

In [29]:
params = [random.random() * 0.01 for _ in range(int(ncas * (ncas - 1) / 2))]

In [148]:
res = minimize(energy_cost_fn, res.x, method='nelder-mead',
               options={'xatol': 1e-30, 'disp': True, 'maxfev': 100}, callback=callback)

Current objective value: -83.80184615111254
Current objective value: -83.80184615111254
Current objective value: -83.80184615111254
Current objective value: -83.80184615111254
Current objective value: -83.80184615111254
Current objective value: -83.80184615111254
Current objective value: -83.80184615111254
Current objective value: -83.80184615111254
Current objective value: -83.80184615111254
Current objective value: -83.80184615111254
Current objective value: -83.80184615111254
Current objective value: -83.80184615111254
Current objective value: -83.80184615111254
Current objective value: -83.80184615111254
Current objective value: -83.80184615111254
Current objective value: -83.80184615111254
Current objective value: -83.80184615111254
Current objective value: -83.80184615111254
Current objective value: -83.80184615111254
Current objective value: -83.80184615111254
Current objective value: -83.80184615111254
Current objective value: -83.80184615111254
Current objective value: -83.801

/var/folders/cm/nzkrh8cx5kq00xj9cvd0zy7w0000gn/T/ipykernel_16128/540551190.py:1: RuntimeWarning: Maximum number of function evaluations has been exceeded.
  res = minimize(energy_cost_fn, res.x, method='nelder-mead',


In [149]:
from pathlib import Path
def transformed_Hamiltonian(params):
    U = construct_orthogonal(ncas, params)
    
    # Get the DMRG energy for BD = 2
    p = np.einsum_path('ak,bl,cm,dn,klmn->abcd', U, U, U, U, combined_tbt_optimizing.copy())[0]
    rotated_tbt = np.einsum('ak,bl,cm,dn,klmn->abcd', U, U, U, U, combined_tbt_optimizing.copy(), optimize = p)
    
    one_body_tensor, two_body_tensor = chemist_to_physicist(obt_dummy, rotated_tbt)
    # label = "_optimized"
    # filename = "fcidump.H4" + f"{label}"
    # pyscf.tools.fcidump.from_integrals(
    #     "fcidump_output"/Path(filename),
    #     one_body_tensor,
    #     two_body_tensor,
    #     nmo=obt_dummy.shape[0],
    #     nelec=n_elec,
    #     nuc=ecore,
    #     ms=spin,
    #     orbsym=None,
    #     tol=1E-8,
    #     float_format=' %.16g',
    # )
    return one_body_tensor, two_body_tensor

In [150]:
obt, tbt = transformed_Hamiltonian(res.x)
# n = obt.shape[0]
# for p in range(n):
#     for q in range(n):
#         if abs(obt[p, q]) < 1e-10:
#             obt[p, q] = 0
#         for r in range(n):
#             for s in range(n):
#                 if abs(tbt[p, q, r, s]) < 1e-10:
#                     tbt[p, q, r, s] = 0

In [153]:
dmrg_params = get_dmrg_config(
    ncas, n_elec, spin,
    2 * spin + 1)
loop_results = dmrg_central_loop(obt, tbt, dmrg_params, 1000, 1e6, 1e-10   , "./mps_directory2", verbosity=1, move_mps_to_final_storage_path=None)

integral cutoff error =  0.0
mpo terms =       4658

Build MPO | Nsites =     7 | Nterms =       4658 | Algorithm = FastBIP | Cutoff = 1.00e-120
 Site =     0 /     7 .. Mmpo =    30 DW = 0.00e+00 NNZ =       30 SPT = 0.0000 Tmvc = 0.000 T = 0.001
 Site =     1 /     7 .. Mmpo =    70 DW = 0.00e+00 NNZ =      329 SPT = 0.8433 Tmvc = 0.000 T = 0.002
 Site =     2 /     7 .. Mmpo =   114 DW = 0.00e+00 NNZ =      597 SPT = 0.9252 Tmvc = 0.000 T = 0.002
 Site =     3 /     7 .. Mmpo =   114 DW = 0.00e+00 NNZ =     2233 SPT = 0.8282 Tmvc = 0.000 T = 0.003
 Site =     4 /     7 .. Mmpo =    70 DW = 0.00e+00 NNZ =      597 SPT = 0.9252 Tmvc = 0.000 T = 0.001
 Site =     5 /     7 .. Mmpo =    30 DW = 0.00e+00 NNZ =      323 SPT = 0.8462 Tmvc = 0.000 T = 0.001
 Site =     6 /     7 .. Mmpo =     1 DW = 0.00e+00 NNZ =       30 SPT = 0.0000 Tmvc = 0.000 T = 0.000
Ttotal =      0.010 Tmvc-total = 0.001 MPO bond dimension =   114 MaxDW = 0.00e+00
NNZ =         4139 SIZE =        33216 SPT = 0.8754

/Users/rick/PycharmProjects/croor/dmrghandler/src/dmrghandler/energy_extrapolation.py:344: RuntimeWarning: divide by zero encountered in log
  a = np.log(alpha)  # α = exp(a), np.log is the natural logarithm


integral cutoff error =  0.0
mpo terms =       4658

Build MPO | Nsites =     7 | Nterms =       4658 | Algorithm = FastBIP | Cutoff = 1.00e-120
 Site =     0 /     7 .. Mmpo =    30 DW = 0.00e+00 NNZ =       30 SPT = 0.0000 Tmvc = 0.000 T = 0.001
 Site =     1 /     7 .. Mmpo =    70 DW = 0.00e+00 NNZ =      329 SPT = 0.8433 Tmvc = 0.000 T = 0.002
 Site =     2 /     7 .. Mmpo =   114 DW = 0.00e+00 NNZ =      597 SPT = 0.9252 Tmvc = 0.000 T = 0.002
 Site =     3 /     7 .. Mmpo =   114 DW = 0.00e+00 NNZ =     2233 SPT = 0.8282 Tmvc = 0.000 T = 0.003
 Site =     4 /     7 .. Mmpo =    70 DW = 0.00e+00 NNZ =      597 SPT = 0.9252 Tmvc = 0.000 T = 0.001
 Site =     5 /     7 .. Mmpo =    30 DW = 0.00e+00 NNZ =      323 SPT = 0.8462 Tmvc = 0.000 T = 0.001
 Site =     6 /     7 .. Mmpo =     1 DW = 0.00e+00 NNZ =       30 SPT = 0.0000 Tmvc = 0.000 T = 0.000
Ttotal =      0.011 Tmvc-total = 0.001 MPO bond dimension =   114 MaxDW = 0.00e+00
NNZ =         4139 SIZE =        33216 SPT = 0.8754

In [152]:
import shutil
import os

# Define the directory path
dir_path = "./mps_directory"

# Check if the directory exists
if os.path.exists(dir_path):
    # Delete the directory and all its contents
    shutil.rmtree(dir_path)
    print(f"Deleted '{dir_path}' and all its contents.")
else:
    print(f"Directory '{dir_path}' does not exist.")
    
# Define the directory path
dir_path = "./mps_directory2"

# Check if the directory exists
if os.path.exists(dir_path):
    # Delete the directory and all its contents
    shutil.rmtree(dir_path)
    print(f"Deleted '{dir_path}' and all its contents.")
else:
    print(f"Directory '{dir_path}' does not exist.")

Directory './mps_directory' does not exist.
Deleted './mps_directory2' and all its contents.
